In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import dm4bem

# Building Project

## Parameters
### Dimension of the Building 

Each Room has a cubic shape

https://github.com/dm4bem-2023/1-model-team7/blob/c375fabf4c8fb06f8ce08dd8fe572f1d180b50a7/Schema.png 

In [2]:
l = 3
Sg = l**2
Sc = Si = 10 * Sg 

We consider that M1,M2,M3 are inside the building and M4 is outside. Then the only exchange are between the inside and the outside with M4 and between the two rooms with M5

### Thermo-Physical Properties
We consider than the wall is made of a width of insulation and a width of concrete (eM=eI+eC)

In [3]:
concrete = {'Conductivity': 1.400,
            'Density': 2400.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 9 * l**2}

insulation = {'Conductivity': 0.027,
              'Density': 55.0,
              'Specific heat': 1210,
              'Width': 0.1,
              'Surface': 9 * l**2}

glass = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 2*(l**2)}

wall = pd.DataFrame.from_dict({'Layer_out': insulation,
                               'Layer_in': concrete,
                               'Glass': glass},
                              orient='index')
wall

air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


### Radiative properties

In [4]:
# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant
print(f'σ = {σ} W/(m²⋅K⁴)')

σ = 5.67e-08 W/(m²⋅K⁴)


### Convection properties

In [5]:
h0 = pd.DataFrame([{'in': 8., 'out': 25}], index=['h0'])  # W/(m²⋅K)
# h = pd.DataFrame([{'in': 8., 'out': 25}])  # W/(m²⋅K)
h0

,in,out
h0,8.0,25


To take into account the long wave radiation, we add 4 at the value of h_in

In [6]:
h = pd.DataFrame([{'in': 12., 'out': 25}], index=['h'])  # W/(m²⋅K)
# h = pd.DataFrame([{'in': 8., 'out': 25}])  # W/(m²⋅K)
h

,in,out
h,12.0,25


### Thermal circuit

Figure bellow shows the models of:

concrete & insulation wall: in red;
glass window: in green;
ventilation: in magenta;
indoor volume: in blue (conductances 6 & 7 for convection; conductance 5 for long wave radiation between the walls and the glass window);
HVAC system: in black.

Here is the thermal circuit :
https://github.com/dm4bem-2023/1-model-team7/blob/c375fabf4c8fb06f8ce08dd8fe572f1d180b50a7/Schema%20thermique.png  

The sources are:

 T0 outdoor temperature, °C;
 Ti,sp setpoint temperaure for the indoor air, °C;
 ϕ0 solar radiation absorbed by the outdoor surface of the wall, W;
 ϕi solar radiation absorbed by the indoor surface of the wall, W;
 Qa auxiliary heat gains (i.e., occupants, electrical devices, etc.), W;
 ϕa solar radiation absorbed by the glass, W.

In [7]:
# Conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

,Conductance
Layer_out,21.87
Layer_in,567.00
Glass,630.00


In [8]:
# Convection wall
Gw = h * wall['Surface'][0]     # wall
print(f'Gw = {Gw} W/K')

Gw =       in   out
h  972.0  2025 W/K


In [9]:
# Convection glass
Gg = h * wall['Surface'][2]     # glass
print(f'Gg = {Gg} W/K')

Gg =       in  out
h  216.0  450 W/K


In [10]:
# Long wave radiation

# view factor wall-glass
Fwg = glass['Surface'] / concrete['Surface']
print(f'Fwg = {Fwg}')

Fwg = 0.2222222222222222


As explained before we do not need these factors because it is included in the value of h_in

In [11]:
# ventilation flow rate
Va = l**3                   # m³, volume of air
ACH = 1                     # air changes per hour
Va_dot = ACH / 3600 * Va    # m³/s, air infiltration
print(f'Va_dot = {Va_dot} m³/s')

Va_dot = 0.0075 m³/s


In [12]:
# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot
print(f'Gv = {Gv} W/K')

Gv = 9.0 W/K


In [13]:
# Proportionnal Controller
# P-controler gain
Kp = 1e6            # almost perfect controller Kp -> ∞
#Kp = 1e-3           # no controller Kp -> 0
#Kp = 0

## Thermal Capacities
### Walls

In [14]:
C = wall['Density'] * wall['Specific heat'] * wall['Surface'] * wall['Width']
C['Air'] = air['Density'] * air['Specific heat'] * Va
pd.DataFrame(C, columns={'Capacity'})

,Capacity
Layer_out,539055.0
Layer_in,34214400.0
Glass,2178000.0
Air,32400.0


## System of algebraic_differential equations (DAE)
### A : Incident Matrix

In [15]:
A = np.zeros([18, 11])       # n° of branches X n° of nodes
A[0, 0] = 1                 # branch 0: -> node 0
A[1, 0], A[1, 1] = -1, 1    # branch 1: node 0 -> node 1
A[2, 1], A[2, 2] = -1, 1    # branch 2: node 1 -> node 2
A[3, 2], A[3, 3] = -1, 1    # branch 3: node 2 -> node 3
A[4, 3], A[4, 4] = -1, 1    # branch 4: node 3 -> node 4
A[5, 4], A[5, 5] = -1, 1    # branch 5: node 4 -> node 5
A[6, 5], A[6, 6] = -1, 1    # branch 6: node 5 -> node 6
A[7, 6], A[7, 7] = -1, 1    # branch 7: node 6 -> node 7
A[8, 7], A[8, 8]= -1, 1     # branch 8: node 7 -> node 8
A[9, 8]= -1                 # branch 9: -> node 8
A[10, 7], A[10, 9]= 1, -1   # branch 10: node 7 -> node 9
A[11, 9], A[11, 6]= 1, -1   # branch 11: node 9 -> node 6
A[12, 10], A[12, 2]= -1, 1  # branch 12: node 10-> node 2
A[13, 1], A[13, 10]= -1, 1  # branch 13: node 1 -> node 10
A[14, 10] = 1               # branch 14: -> node 10
A[15, 10] = 1               # branch 15: -> node 10
A[16, 9] = 1                # branch 16: -> node 9
A[17, 9] = 1                # branch 17: -> node 9

# np.set_printoptions(suppress=False)
# pd.DataFrame(A)

### G : Incident Matrix

In [16]:
G = np.zeros([18, 18])       # n° of branches X n° of nodes
G[0, 0] = (h[1]*glass['Surface'])/2                  
G[1, 1] = (glass['Conductivity']*glass['Surface']/2)/glass['Width']
G[2, 2] = 0
G[3, 3] = (concrete['Conductivity']*concrete['Surface']/2)/concrete['Width']
G[4, 4] = (concrete['Conductivity']*concrete['Surface']/2)/concrete['Width']              
G[5, 5] = (insulation['Conductivity']*insulation['Surface']/2)/insulation['Width']
G[6, 6] = (insulation['Conductivity']*insulation['Surface']/2)/insulation['Width']
G[7, 7] = 0
G[8, 8] = (glass['Conductivity']*glass['Surface']/2)/glass['Width']                  
G[9, 9] = (h[1]*glass['Surface'])/2 
G[10, 10] = (h[0]*glass['Surface'])/2  
G[11, 11] = (h[0]*concrete['Surface'])/2 
G[12, 12] = (h[0]*concrete['Surface'])/2                   
G[13, 13] = (h[0]*glass['Surface'])/2 
G[14, 14] = Gv 
G[15, 15] = Kp
G[16, 16] = Kp                  
G[17, 17] = Gv

KeyError: 1

### C : Capacity Matrix

In [ ]:
neglect_air_glass = False

if neglect_air_glass:
    C = np.diag([0, 0, 0, C['Layer_out'], 0, C['Layer_in'], 0, 0,
                 0, 0, 0])
else:
    C = np.diag([C['Glass'], 0, 0, C['Layer_out'], 0, C['Layer_in'], 0, 0, C['Glass'],
                 C['Air'], C['Air']])

# pd.set_option("display.precision", 3)
# pd.DataFrame(C)

### b : temperature source vector 

In [ ]:
b = np.zeros(18)        # branches
b[[0, 9, 14, 15, 16, 17]] = 1   # branches with temperature sources
print(f'b = ', b)

### f : heat flow source  vector 

In [ ]:
f = np.zeros(11)         # nodes
f[[0, 2, 6, 7, 8, 9, 10]] = 1     # nodes with heat-flow sources
print(f'f = ', f)

### y : output vector 

In [ ]:
y = np.zeros(11)         # nodes
y[[9, 10]] = 1              # nodes (temperatures) of interest
print(f'y = ', y)

## State-space representation


In [ ]:
#[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
#print('As = \n', As, '\n')
#print('Bs = \n', Bs, '\n')
#print('Cs = \n', Cs, '\n')
#print('Ds = \n', Ds, '\n')

